# The Db2 Data Management Console Technical Preview 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, userid, password, profile=""):
        credentials = {'userid':userid, 'password':password}
        r = requests.post(self.url+'/auth/tokens', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = r.json()['token']
            if profile == "":
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken}
            else:
                self.headers = {'Authorization': 'Bearer'+ ' '+bearerToken, 'X-DB-Profile': profile}
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getSchemas(self):
        return self.getRequest('/schemas')
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/sql_jobs/'+jobid)
    
    def getCurrentApplicationsConnections(self):
        return self.getRequest('/metrics/applications/connections/current/list')
    
    def getInflightCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements/inflight_executions/current/list?start='+str(startTime)+'&end='+str(endTime));
       
    def getInflightCurrentList(self):
        return self.getRequest('/metrics/statements/inflight_executions/current/list');
    
    def getIndividualStatementExecution(self, startTime, endTime, limit=100, includeSystem='false'):
        return self.getRequest('/metrics/statements/evmon_activity?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem)+'&offset=0&limit='+str(limit))

    def getFiles(self, path):
        return self.getRequest('/home'+path)
    
    def getUsers(self):
        return self.getRequest('/users')
    
    def getTablesMetrics(self, startTime, endTime, includeSystem='false'):
        return self.getRequest('/metrics/tables?start='+str(startTime)+'&end='+str(endTime)+'&include_sys='+str(includeSystem));

    def getAverageResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/average_response_time?start='+str(startTime)+'&end='+str(endTime));    
    
    def getUnitsOfWork(self, startTime, endTime):
        return self.getRequest('/applications/uow?start='+str(startTime)+'&end='+str(endTime));    
    
    def getSchemaSize(self, startTime, endTime, tabSchema):
        return self.getRequest('/metrics/storage/schemas/'+tabSchema+'/timeseries?start='+str(startTime)+'&end='+str(endTime));
  
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
              
    def getRowsRead(self, startTime, endTime):
        return self.getRequest('/metrics/rows_read?start='+str(startTime)+'&end='+str(endTime));

    def getResponseTime(self, startTime, endTime):
        return self.getRequest('/metrics/response_time?start='+str(startTime)+'&end='+str(endTime));

    def getStatementsCount(self, startTime, endTime):
        return self.getRequest('/metrics/statements_count?start='+str(startTime)+'&end='+str(endTime));

    def getPackageCacheStatement(self, startTime, endTime):
        return self.getRequest('/metrics/statements/package_cache?start='+str(startTime)+'&end='+str(endTime))
    
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/admin/'+str(obj_type)+'s',json);
                
    def getEXPLAINModel(self, profile, currentSchema, statement):
        json = {'schema':currentSchema, 'sql':statement}
        explainHeaders = self.headers.copy()
        explainHeaders['X-DB-Profile'] = profile
        return requests.post(self.url+'/explain/explain_model', verify = self.verify, headers=explainHeaders, proxies = self.proxies, json=json) 
    
    def getEXPLAINNodes(self, profile, session):
        explainHeaders = self.headers.copy()
        explainHeaders['X-DB-Profile'] = profile
        return requests.get(self.url+'/explain/explain_model/'+session+'/diagram/nodes', verify = self.verify, headers=explainHeaders, proxies = self.proxies) 

    def getEXPLAINLayout(self, profile, session, ids, labels):
        idsWithWidth = list()
        labelsWithWidth = list()
        for id in ids:
            idsWithWidth.append({"text":id, "width": 60})
        for label in labels:
            labelsWithWidth.append({"text": label, "width": 60})
        json = {"idFontHeight":20, "ids": idsWithWidth, "labelFontHeight":20, "label1s": labelsWithWidth}
        pprint(json)
        explainHeaders = self.headers.copy()
        explainHeaders['X-DB-Profile'] = profile
        return requests.post(self.url+'/explain/explain_model/'+session+'/diagram/layout', verify = self.verify, headers=explainHeaders, proxies = self.proxies, json=json) 
    
    def putFile(self, filename, path):
        with open(filename, 'rb') as f:
            r = requests.post(self.url+'/home_content/path', files={filename: f}, verify = self.verify, headers=self.headers, proxies = self.proxies)
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/schemas/'+str(schema)+'/tables');
    
    def getTableStorageBySchema(self):
        return self.getRequest('/monitor/storage/by_schema');
    
    def getCurrentPackageCacheList(self):
        return self.getRequest('/metrics/statements/package_cache/current/list')
    
    def getProfile(self,profile):
        return self.getRequest('/dbprofiles/'+profile)    

In [3]:
# Setup data frame set calculation functions
def epochtotimeseries(epoch):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch/1000))
def KBtoGB(kb):
    return kb/1024/1024

In [4]:
# Setup time series calculation values
import time
from datetime import date
endTime = int(time.time())
endTime = endTime*1000
startTime = endTime-(600*1000)
oneWeek = 604800000

In [5]:
# Increase the width of the notebook working spaces
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# CLOUD Databases
#database = Db2("https://158.175.106.46/dbapi/v4", False, {'https': 'socks5://127.0.0.1:4444'})
#database = Db2("https://158.175.106.46/dbapi/v4")
#database = Db2("https://9.30.146.214/dbapi/v4")
#database.authenticate("blutestadmin", "Password!234")

In [10]:
# Enterprise Console Playground
profile = "testConn"
profileURL = "?profile="+profile
database = 'http://9.111.141.96:11080'
databaseAPI = Db2(database+'/dbapi/v4')
databaseAPI.authenticate("admin", "passw0rd", profile)

In [7]:
# Enterprise Console Local
profile = "SAMPLE"
profileURL = "?profile="+profile
database = 'https://127.0.0.1:11081'
databaseAPI = Db2(database+'/dbapi/v4')
databaseAPI.authenticate("admin", "password", profile)

In [ ]:
# BIGSQL Database
profileURL = ""
database = 'https://9.30.160.36:11081'
databaseAPI = Db2(database+'/dbapi/v4')
databaseAPI.authenticate("bigsql", "bigsql")

In [11]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

{'name': 'testConn', 'HADR_Role': 'STANDARD', 'disableDataCollection': 'false', 'databaseVersion': '11.1.3', 'databaseName': 'sample', 'timeZone': '80000', 'isHADR': 'false', 'DB2Instance': 'db2inst1', 'db2license': 'AESE', 'isInstPureScale': 'false', 'databaseVersion_VRMF': '11.1.3.3', 'sslConnection': 'false', 'userProfileRole': 'OWNER', 'timeZoneDiff': '0', 'host': '9.110.83.221', '_PROFILE_INIT_': 'true', 'dataServerType': 'DB2LUW', 'port': '50000', 'URL': 'jdbc:db2://9.110.83.221:50000/sample', 'edition': 'AESE', 'isInstPartitionable': 'false', 'dataServerExternalType': 'DB2LUW', 'CF_NODE': '', 'capabilities': '["trial"]', 'OSType': 'Linux'}


In [12]:
# List all schema
r = databaseAPI.getSchemas()
json = databaseAPI.getJSON(r)
df = pd.DataFrame(json_normalize(json['resources']))
display(df[['name']].head(10)) 

,name
0,CINDY
1,DB2INST1
2,IBMCONSOLE
3,IBM_RTMON
4,NULLID
5,SQLJ
6,SYSCAT
7,SYSFUN
8,SYSIBM
9,SYSIBMADM


In [13]:
# List all schema
r = databaseAPI.getSchemas()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

definertype, name


,name
0,CINDY
1,DB2INST1
2,IBMCONSOLE
3,IBM_RTMON
4,NULLID
5,SQLJ
6,SYSCAT
7,SYSFUN
8,SYSIBM
9,SYSIBMADM


In [14]:
IFrame(database+'/console/?mode=compact#monitor/summary'+profileURL, width=1400, height=1000)

In [15]:
# Retrieve the number of rows read
# List the last 10 data points
# Graph the history of the storage usage
r = databaseAPI.getRowsRead(0, endTime)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:
        df = pd.DataFrame(json_normalize(json['timeseries']))
        print('Columns')
        print(', '.join(list(df)))
        df['timestamp'] = df['timestamp'].apply(epochtotimeseries)
        display(df[['timestamp','rows_read_per_min']].tail(20))
        df.plot(x='timestamp',y='rows_read_per_min') 
    else: 
        print('No data returned')
else:
    print(databaseAPI.getStatusCode(r))
    


No data returned


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/overview'+profileURL, width=1600, height=1200)

In [ ]:
IFrame(database+'/console/?mode=compact#sql/explain/e015be3b-2849-4ed4-ab9a-478c1cea38bc', width=1600, height=700)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/inflight_executions'+profileURL, width=1400, height=360)

In [16]:
# Retreive the current statements running now
# Display the top 10 by execution time
r = databaseAPI.getInflightCurrentList()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='exec_time_ms', ascending=False)
        display(df[['application_name','stmt_text','exec_time_ms','estimated_runtime_ms']].head(10))
    else: 
        print('No data returned')
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

No data returned


In [ ]:
IFrame(database+'/console/?mode=compact#monitor/individual'+profileURL, width=1400, height=480)

In [ ]:
# Retrieve the statements that ran over the last week.
# Retrieve the first 10000 statements
# Show the top ten by Total CPU TIME
startTimeMinusWeek = endTime - oneWeek
r = databaseAPI.getIndividualStatementExecution(startTimeMinusWeek, endTime, 10000)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:     
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='total_cpu_time', ascending=False)
        display(df[['sql_text','total_cpu_time','wlm_queue_time_total','stmt_exec_time']].head(10))
    else: 
        print('No data returned')  
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/package_cache'+profileURL, width=1400, height=480)

In [ ]:
r = databaseAPI.getCurrentPackageCacheList()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:  
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        display(df[['stmt_text','stmt_exec_time_ms','stmtid']].head(10))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/locked_objects_with_waiting_connections'+profileURL, width=1400, height=480)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/connections'+profileURL, width=1400, height=480)

In [ ]:
# Display the 10 most recently started Database Connections
r = databaseAPI.getCurrentApplicationsConnections()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['resources']))
        print(', '.join(list(df)))
        df = df.sort_values(by='connection_start_time', ascending=False)
        df['connection_start_time'] = df['connection_start_time'].apply(epochtotimeseries)
        display(df[['application_name','application_handle','connection_start_time']].head(10))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/wlm_workload_summary', width=1400, height=480)

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/storage', width=1400, height=480)

In [ ]:
# Retrieve the storage size for the Historical Data Repostory stored in IBMCONSOLE schema.
# List the last 10 data points
# Graph the history of the storage usage
tabSchema = 'IBMCONSOLE'
r = databaseAPI.getSchemaSize(0, endTime, tabSchema)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['timeseries']))
        print('Columns')
        print(', '.join(list(df)))
        df['total_physical_size_gb'] = df['total_physical_size_kb'].apply(KBtoGB)
        df['timestamp'] = df['timestamp'].apply(epochtotimeseries)
        display(df[['timestamp','total_physical_size_gb']].tail(10))
        df.plot(x='timestamp',y='total_physical_size_gb')
    else: 
        print('No data returned') 
else:
    print(databaseAPI.getStatusCode(r))

In [ ]:
# List storage used by schema
# Display the top ten schemas
r = databaseAPI.getTableStorageBySchema()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['resources']))
        df['space_mb'] = df['data_physical_size_kb'].apply(lambda x: x / 1024)
        df = df.sort_values(by='data_physical_size_kb', ascending=False)    
        display(df[['schema','space_mb']].head(10))
    else: 
        print('No data returned') 
else:
    print("RC: "+str(databaseAPI.getStatusCode(r)))    

In [ ]:
# List any open alerts using part of the Console Interface
IFrame(database+'/console/?mode=compact#monitor/notification'+profileURL, width=1400, height=480)

In [ ]:
# Explore tables in your database using part of the Console Interface
IFrame(database+'/console/?mode=compact#explore/table'+profileURL, width=1400, height=480)

In [17]:
# Search for tables across all schemas that match simple search critera 
# Display the first 100
# Switch between searching tables or views
object = 'view'
#object = 'table'
r = databaseAPI.postSearchObjects(object,"TABLE",10,'true','false')
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print('Columns:')
    print(', '.join(list(df)))
    display(df[[object+'_name']].head(100))
else:
    print("RC: "+str(databaseAPI.getStatusCode(r)))

Columns:
alter_time, create_time, optimize_query, owner, owner_type, read_only, sql, stats_time, valid, view_check, view_name, view_schema


,view_name
0,TABLE_CONSTRAINTS
1,TABLES
2,TABLES_S
3,EVENTTABLES
4,TABLES
5,TABLESPACES
6,TABLES
7,SQLTABLEPRIVILEGES
8,SQLTABLETYPES
9,SQLTABLES


In [18]:
# Find all the tables in the SYSIBM schema and display the first 10
schema = 'SYSIBM'
r = databaseAPI.getTablesInSchema(schema)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['schema','name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

name, schema


,schema,name
0,SYSIBM,SYN190527163358641001_T2
1,SYSIBM,SYSATTRIBUTES
2,SYSIBM,SYSAUDITEXCEPTIONS
3,SYSIBM,SYSAUDITPOLICIES
4,SYSIBM,SYSAUDITUSE
5,SYSIBM,SYSBUFFERPOOLNODES
6,SYSIBM,SYSBUFFERPOOLS
7,SYSIBM,SYSCHECKS
8,SYSIBM,SYSCODEPROPERTIES
9,SYSIBM,SYSCOLAUTH
